# `imbrium` use case
## Hyper parameter optimization - Bidirectional Long Short-term Neural Network

### Multivariate pure forecasting

##### Example Steps:

- basic data preparation
- scale target and feature numpy arrays
- create imbrium bidirectional long short-term neural network
- use optuna to perform hyper parameter optimzation on shifting window variables

In [1]:
import imbrium
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import optuna

print(f"imbrium version: {imbrium.__version__} loaded")

imbrium version: 3.0.0 loaded


In [2]:
example_data = pd.read_csv('example_dataset/AirQualityUCI.csv', delimiter=';')
example_data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [3]:
example_data = example_data[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [4]:
example_data.isna().sum()

CO(GT)           114
PT08.S1(CO)      114
NMHC(GT)         114
C6H6(GT)         114
PT08.S2(NMHC)    114
dtype: int64

In [5]:
example_data = example_data.dropna()

In [6]:
example_data.notna().sum()

CO(GT)           9357
PT08.S1(CO)      9357
NMHC(GT)         9357
C6H6(GT)         9357
PT08.S2(NMHC)    9357
dtype: int64

In [7]:
example_data = example_data.replace(",", ".", regex = True).astype("float")

In [8]:
target = np.array(example_data['PT08.S1(CO)']).reshape(-1, 1)

In [9]:
features = example_data[['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)']]

In [10]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

scaled_features = feature_scaler.fit_transform(features)
scaled_target = target_scaler.fit_transform(target)

In [11]:
custom_layer_config = {
            "layer0": {
                "config": {
                    "neurons": 200,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer1": {
                "config": {
                    "neurons": 100,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer2": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.2,
                }
            },
            "layer3": {
                "config": {
                    "neurons": 50,
                    "activation": "relu",
                    "regularization": 0.002,
                    "dropout": 0.002
                }
            },
            "layer4": {
                "config": {
                    "neurons": 25,
                    "activation": "relu",
                    "regularization": 0.002,
                }
            },
        }

In [12]:
from imbrium import PureMulti

In [13]:
def objective(trial):

    predictor_instance = PureMulti(target = scaled_target, features = scaled_features)

    steps_past = trial.suggest_int('steps_past', 5, 30)
    steps_future = trial.suggest_int('steps_future', 5, 10)
    
    
    metric_value = predictor_instance.create_fit_bilstm(
        steps_past =  steps_past,
        steps_future = steps_future,
        loss='mean_squared_error',
        metrics='mean_squared_error',
        bilstm_block_one = 3,
        lstm_block_one = 2,
        layer_config = custom_layer_config, 
        epochs=10,
        show_progress=1,
        board=False,
        monitor='val_loss',
        patience=3,
        min_delta=0,
        verbose=1
    )


    return metric_value

study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=5) 

[I 2024-05-24 01:18:10,457] A new study created in memory with name: no-name-1affb4f0-95ce-455a-8aaa-ac13b84e77c2



Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 18s 48ms/step - loss: 1.3250 - mean_squared_error: 0.1224 - val_loss: 0.1162 - val_mean_squared_error: 0.0285
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0883 - mean_squared_error: 0.0193 - val_loss: 0.0657 - val_mean_squared_error: 0.0292
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 0.0511 - mean_squared_error: 0.0192 - val_loss: 0.0561 - val_mean_squared_error: 0.0349
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - loss: 0.0402 - mean_squared_error: 0.0208 - val_loss: 0.0487 - val_mean_squared_error: 0.0345
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - loss: 0.0351 - mean_squared_error: 0.0216 - val_loss: 0.0440 - val_mean_squared_error: 0.0338
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - loss: 0.1264 - mean_squared_error: 0.1157 - val_loss: 0.0744 - val_mean_squared_error: 0.0402
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - loss: 0.0506 - mean_squared_error: 0.0213 - val_l

[I 2024-05-24 01:19:33,012] Trial 0 finished with value: 0.0169202983379364 and parameters: {'steps_past': 12, 'steps_future': 6}. Best is trial 0 with value: 0.0169202983379364.


Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 25s 81ms/step - loss: 1.4008 - mean_squared_error: 0.1716 - val_loss: 0.1995 - val_mean_squared_error: 0.0325
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 70ms/step - loss: 0.1627 - mean_squared_error: 0.0207 - val_loss: 0.1252 - val_mean_squared_error: 0.0330
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - loss: 0.1035 - mean_squared_error: 0.0210 - val_loss: 0.0926 - val_mean_squared_error: 0.0332
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 70ms/step - loss: 0.0745 - mean_squared_error: 0.0204 - val_loss: 0.0741 - val_mean_squared_error: 0.0333
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - loss: 0.0580 - mean_squared_error: 0.0204 - val_loss: 0.0621 - val_mean_squared_error: 0.0329
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - loss: 0.0473 - mean_squared_error: 0.0200 - val_loss: 0.0537 - val_mean_squared_error: 0.0320
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - loss: 0.0493 - mean_squared_error: 0.0280 - 

[I 2024-05-24 01:21:45,020] Trial 1 finished with value: 0.01678040809929371 and parameters: {'steps_past': 13, 'steps_future': 9}. Best is trial 1 with value: 0.01678040809929371.


...

[I 2024-05-24 01:23:46,759] Trial 2 finished with value: 0.01756777986884117 and parameters: {'steps_past': 25, 'steps_future': 7}. Best is trial 1 with value: 0.01678040809929371.


Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step - loss: 1.3230 - mean_squared_error: 0.1327 - val_loss: 0.1024 - val_mean_squared_error: 0.0359
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - loss: 0.0745 - mean_squared_error: 0.0230 - val_loss: 0.0796 - val_mean_squared_error: 0.0509
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 70ms/step - loss: 0.0487 - mean_squared_error: 0.0239 - val_loss: 0.0562 - val_mean_squared_error: 0.0385
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 69ms/step - loss: 0.0389 - mean_squared_error: 0.0219 - val_loss: 0.0442 - val_mean_squared_error: 0.0308
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 71ms/step - loss: 0.0307 - mean_squared_error: 0.0185 - val_loss: 0.0777 - val_mean_squared_error: 0.0663
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - loss: 0.0508 - mean_squared_error: 0.0368 - val_loss: 0.0587 - val_mean_squared_error: 0.0372
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 14s 75ms/step - loss: 0.0403 - mean_squared_error: 0.0208 - 

[I 2024-05-24 01:25:28,806] Trial 3 finished with value: 0.017661530524492264 and parameters: {'steps_past': 15, 'steps_future': 9}. Best is trial 1 with value: 0.01678040809929371.


Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - loss: 1.3311 - mean_squared_error: 0.1256 - val_loss: 0.1327 - val_mean_squared_error: 0.0364
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - loss: 0.0979 - mean_squared_error: 0.0218 - val_loss: 0.0779 - val_mean_squared_error: 0.0384
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - loss: 0.0570 - mean_squared_error: 0.0225 - val_loss: 0.0571 - val_mean_squared_error: 0.0347
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - loss: 0.0417 - mean_squared_error: 0.0214 - val_loss: 0.0480 - val_mean_squared_error: 0.0334
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - loss: 6.0274 - mean_squared_error: 6.0115 - val_loss: 0.0639 - val_mean_squared_error: 0.0411
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - loss: 0.0405 - mean_squared_error: 0.0204 - val_loss: 0.0515 - val_mean_squared_error: 0.0351
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - loss: 0.0368 - mean_squared_error: 0.0209 - 

[I 2024-05-24 01:26:57,873] Trial 4 finished with value: 0.01698741689324379 and parameters: {'steps_past': 11, 'steps_future': 7}. Best is trial 1 with value: 0.01678040809929371.


In [14]:
best_params = study.best_params
best_params

{'steps_past': 13, 'steps_future': 9}

In [15]:
predictor_instance = PureMulti(target = scaled_target, features = scaled_features)

predictor_instance.create_fit_bilstm(
        steps_past =  best_params["steps_past"],
        steps_future = best_params["steps_future"],
        loss='mean_squared_error',
        metrics='mean_squared_error',
        bilstm_block_one = 3,
        lstm_block_one = 2,
        layer_config = custom_layer_config, 
        epochs=5,
        show_progress=1,
        board=False,
        monitor='val_loss',
        patience=3,
        min_delta=0,
        verbose=1
    )

Epoch 1/5
187/187 ━━━━━━━━━━━━━━━━━━━━ 23s 71ms/step - loss: 1.3235 - mean_squared_error: 0.1442 - val_loss: 0.0924 - val_mean_squared_error: 0.0285
Epoch 2/5
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - loss: 0.0699 - mean_squared_error: 0.0201 - val_loss: 0.0629 - val_mean_squared_error: 0.0365
Epoch 3/5
187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - loss: 0.0450 - mean_squared_error: 0.0215 - val_loss: 0.0551 - val_mean_squared_error: 0.0390
Epoch 4/5
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 66ms/step - loss: 0.0363 - mean_squared_error: 0.0217 - val_loss: 0.0451 - val_mean_squared_error: 0.0340
Epoch 5/5
187/187 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - loss: 0.0317 - mean_squared_error: 0.0214 - val_loss: 0.0441 - val_mean_squared_error: 0.0361


0.016952170059084892

In [16]:
predictor_instance.evaluate_model()
metric_value = predictor_instance.show_evaluation()[1]

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0331 - mean_squared_error: 0.0252


In [17]:
metric_value

0.023533688858151436